In [1]:
from sympy import *
from sympy.physics.quantum import *
from sympy.core.numbers import I, One
from sympy.simplify.fu import *
from IPython.display import display

In [2]:
sigma_x = Matrix([[0,1],[1,0]])
sigma_y = Matrix([[0,-I],[I,0]])
sigma_z = Matrix([[1,0],[0,-1]])
one = Identity(2).as_mutable()
sx = sigma_x/2
ix = sigma_x/2
sy = sigma_y/2
iy = sigma_y/2
iy = sigma_y/2
sz = sigma_z/2
iz = sigma_z/2

In [3]:
class ProductOperator(UnitaryOperator):

    def __new__(cls, label, *args, matrix, **kwargs):
        inst = super().__new__(cls, label, *args, **kwargs)
        inst.matrix = matrix
        return inst
    
    def __str__(self):
        display(*self.label)
        return ''
    
    def doit(self):
        return self.matrix
    
    def norm(self):
        norm = sqrt((self.matrix*Dagger(self.matrix)).trace())
        return norm
    
    @property
    def is_normalized(self):
        return True if self.norm()==1 else False

    def as_normalized(self):
        return self/self.norm()


In [4]:
One = ProductOperator('I', matrix=TensorProduct(one,one))
Sx = ProductOperator('S_x', matrix=TensorProduct(sx,one))
Sy = ProductOperator('S_y', matrix=TensorProduct(sy,one))
Sz = ProductOperator('S_z', matrix=TensorProduct(sz,one))
Ix = ProductOperator('I_x', matrix=TensorProduct(one,ix))
Iy = ProductOperator('I_y', matrix=TensorProduct(one,iy))
Iz = ProductOperator('I_z', matrix=TensorProduct(one,iz))
SzIx = ProductOperator('S_{z}I_{x}', matrix=TensorProduct(sz,ix))
SzIy = ProductOperator('S_{z}I_{y}', matrix=TensorProduct(sz,iy))
SxIz = ProductOperator('S_{x}I_{z}', matrix=TensorProduct(sx,iz))
SyIz = ProductOperator('S_{y}I_{z}', matrix=TensorProduct(sy,iz))
SzIz = ProductOperator('S_{z}I_{z}', matrix=TensorProduct(sz,iz))
SxIx = ProductOperator('S_{x}I_{x}', matrix=TensorProduct(sx,ix))
SyIy = ProductOperator('S_{y}I_{y}', matrix=TensorProduct(sy,iy))
SxIy = ProductOperator('S_{x}I_{y}', matrix=TensorProduct(sx,iy))
SyIx = ProductOperator('S_{y}I_{x}', matrix=TensorProduct(sy,ix))

basis_set = [One, Sx, Sy, Sz, Ix, Iy, Iz, SzIx, SzIy, SxIz, SyIz,
             SzIz, SxIx, SyIy, SxIy, SyIx]

In [5]:
Omega_S, omega_I, a_iso, A, B, t = symbols('Omega_S omega_I a_iso A B t', real=True)

In [6]:
def get_coef(matrix, base:ProductOperator):
    base_norm = base.norm()
    if isinstance(matrix, MatrixBase):
        coef = (base.as_normalized().doit()*matrix).trace()
    else:
        coef = (base.as_normalized()*matrix).doit().trace()
    coef = coef/base_norm  
    coef = simplify(coef, rational=True)     
    return coef

In [7]:
class DensityMatrix(ProductOperator):

    def __new__(cls, label, *args, matrix, **kwargs):
        tmp_inst = super().__new__(cls, label, *args, matrix=matrix, **kwargs)
        new_label = tmp_inst.operators_repr()
        inst = super().__new__(cls, new_label, *args, matrix=matrix, **kwargs)
        return inst
   
    def operators_repr(self):
        rep = 0
        for base in basis_set:
            coef = get_coef(self.matrix, base)
            rep += base*coef
        return rep


In [8]:
def spin_evolve(A:ProductOperator, B:ProductOperator, phi):
    """
    Evolution of spin state
    Args:
        A: initial state
        B: Hamiltonian
        phi: rotation angle
    """ 
    angle = phi*nsimplify(B.norm())
    res = A*cos(angle) + I*Commutator(A,B.as_normalized())*sin(angle)
    res_matrix = res.doit()
    res_matrix = simplify(res_matrix, rational=True)
    ret = DensityMatrix('sigma', matrix=res_matrix)
    return ret

In [9]:
s1 = spin_evolve(Sy, Sz, Omega_S*t)
s1

-sin(Omega_S*t)*S_x + cos(Omega_S*t)*S_y

In [10]:
s1.doit()

Matrix([
[                                     0,                                      0, -sin(Omega_S*t)/2 - I*cos(Omega_S*t)/2,                                      0],
[                                     0,                                      0,                                      0, -sin(Omega_S*t)/2 - I*cos(Omega_S*t)/2],
[-sin(Omega_S*t)/2 + I*cos(Omega_S*t)/2,                                      0,                                      0,                                      0],
[                                     0, -sin(Omega_S*t)/2 + I*cos(Omega_S*t)/2,                                      0,                                      0]])

In [11]:
s2 = spin_evolve(s1, SzIz, a_iso*t)
s2

-2*sin(Omega_S*t)*sin(a_iso*t/2)*S_{y}I_{z} - sin(Omega_S*t)*cos(a_iso*t/2)*S_x - 2*sin(a_iso*t/2)*cos(Omega_S*t)*S_{x}I_{z} + cos(Omega_S*t)*cos(a_iso*t/2)*S_y

In [12]:
s2.doit()

Matrix([
[                                                             0,                                                              0, -sin(t*(Omega_S + a_iso/2))/2 - I*cos(t*(Omega_S + a_iso/2))/2,                                                              0],
[                                                             0,                                                              0,                                                              0, -sin(t*(Omega_S - a_iso/2))/2 - I*cos(t*(Omega_S - a_iso/2))/2],
[-sin(t*(Omega_S + a_iso/2))/2 + I*cos(t*(Omega_S + a_iso/2))/2,                                                              0,                                                              0,                                                              0],
[                                                             0, -sin(t*(Omega_S - a_iso/2))/2 + I*cos(t*(Omega_S - a_iso/2))/2,                                                              0,                         

In [13]:
eta_alpha, eta_beta, eta, xi = symbols('eta_alpha eta_beta eta xi', real=True)
omega_12, omega_34, omega_plus, omega_minus = symbols('omega_12, omega_34, omega_+, omega_-', real=True)

constr1 = Eq(eta_alpha, atan(-B/(A+2*omega_I)))
constr2 = Eq(eta_beta, atan(-B/(A-2*omega_I)))

In [14]:
eta_alpha = atan(-B/(A+2*omega_I))
eta_beta = atan(-B/(A-2*omega_I))
xi = (eta_alpha+eta_beta)/2
eta = (eta_alpha-eta_beta)/2
omega_12 = (omega_I+A/2)*cos(eta_alpha) - B/2*sin(eta_alpha)
omega_34 = (omega_I-A/2)*cos(eta_beta) + B/2*sin(eta_beta)
omega_plus = omega_12 + omega_34
omega_minus = omega_12 - omega_34

In [15]:
H0 = Omega_S*Sz + omega_I*Iz + A*SzIz + B*SzIx
H0.doit()

Matrix([
[A/4 + Omega_S/2 + omega_I/2,                          B/4,                            0,                           0],
[                        B/4, -A/4 + Omega_S/2 - omega_I/2,                            0,                           0],
[                          0,                            0, -A/4 - Omega_S/2 + omega_I/2,                        -B/4],
[                          0,                            0,                         -B/4, A/4 - Omega_S/2 - omega_I/2]])

In [24]:
Uexpo = -I*(xi*Iy+2*eta*SzIy).doit()

In [26]:
U = exp(Uexpo)